# 03 — 將資料集上傳到 Hugging Face Hub（Dataset Repo）
<div align="left" style="line-height: 1;">
  <a href="https://discord.gg/Cx737yw4ed" target="_blank" style="margin: 2px;">
    <img alt="Discord" src="https://img.shields.io/badge/Discord-Twinkle%20AI-7289da?logo=discord&logoColor=white&color=7289da" style="display: inline-block; vertical-align: middle;"/>
  </a>
  <a href="https://huggingface.co/twinkle-ai" target="_blank" style="margin: 2px;">
    <img alt="Hugging Face" src="https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Twinkle%20AI-ffc107?color=ffc107&logoColor=white" style="display: inline-block; vertical-align: middle;"/>
  </a>
      <a href="https://colab.research.google.com/github/ai-twinkle/llm-lab/blob/main/courses/2025-08-llm-dialogue-dataset/03_upload_to_hfhub.ipynb" target="_blank" style="margin: 2px;">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open 03_format_for_training In Colab" style="display: inline-block; vertical-align: middle;"/>
  </a>
</div>

本章目標：
1. 準備要上傳的檔案（預期：`outputs/datasets.jsonl`）
2. 使用 `huggingface_hub` 建立或覆用 **Dataset repo**
3. 上傳 `data/train.jsonl`（選配：同時上傳 `train.parquet`）
4. 建立 / 更新 Dataset Card（`README.md`）

In [ ]:
!pip -q install huggingface_hub datasets pyarrow
!curl -L https://raw.githubusercontent.com/ai-twinkle/llm-lab/main/courses/2025-08-llm-dialogue-dataset/outputs/datasets.jsonl -o datasets.jsonl

from huggingface_hub import HfApi, create_repo, upload_file, upload_folder
from huggingface_hub import login as hf_login
from pathlib import Path
import json, os, time

# === 基本設定（請依實際調整） ===
HF_TOKEN = ""  # ← 向 Twinkle AI 社群索取（請勿上傳到公開 repo）
ORG_OR_USER = "tw-llama"  # 你的組織/個人帳號
DATASET_NAME = "twinkle-dialogue-gemma3-2025-08"  # 建議有日期與主題
REPO_ID = f"{ORG_OR_USER}/{DATASET_NAME}"         # 例：ai-twinkle/twinkle-dialogue-gemma3-2025-08

LOCAL_JSONL = Path("outputs/datasets.jsonl")      # 01/02 章節累積的主檔，註：這裡仍先以 datasets.jsonl，使用者可以再考慮要不要上傳 clean.jsonl
assert LOCAL_JSONL.exists(), f"找不到 {LOCAL_JSONL}，請先完成前面章節生成資料"

# 可選：是否也上傳 Parquet（HF Hub 也會在後台自動生成 parquet 分支，但這裡示範手動輸出一次）
ALSO_UPLOAD_PARQUET = True

print("Repo:", REPO_ID)
print("Local file:", LOCAL_JSONL.resolve())

In [ ]:
CARD_PATH = Path("outputs/README.md")
CARD_PATH.parent.mkdir(parents=True, exist_ok=True)

# 注意：HF 會讀取 README.md 頂端的 YAML 區塊作為中繼資料
card_md = f"""---
pretty_name: Twinkle Dialogue (Gemma-3-12B-it, 2025-08)
tags:
- dialog
- instruction-tuning
- sft
- openai-messages
license: cc-by-4.0
task_categories:
- text-generation
dataset_info:
  features:
  - name: messages
    sequence: 
      - name: role
        dtype: string
      - name: content
        dtype: string
  splits:
  - name: train
    num_bytes: 123456
    num_examples: 1000
  download_size: 123456
  dataset_size: 123456
language:
- zh
---

# Twinkle Dialogue (Gemma-3-12B-it, 2025-08)
<div align="left" style="line-height: 1;">
  <a href="https://discord.gg/Cx737yw4ed" target="_blank" style="margin: 2px;">
    <img alt="Discord" src="https://img.shields.io/badge/Discord-Twinkle%20AI-7289da?logo=discord&logoColor=white&color=7289da" style="display: inline-block; vertical-align: middle;"/>
  </a>
  <a href="https://huggingface.co/twinkle-ai" target="_blank" style="margin: 2px;">
    <img alt="Hugging Face" src="https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Twinkle%20AI-ffc107?color=ffc107&logoColor=white" style="display: inline-block; vertical-align: middle;"/>
  </a>
</div>

本資料集由 **Gemma-3-12B-it（Twinkle AI 社群服務）** 生成之對話資料，採用 **OpenAI Chat Messages** 格式（`.jsonl`），並整合：
- Reference-free（由 seed 派生單輪問答）
- Reference-based（依據參考文本生成單輪問答）

> 檔案路徑：`data/train.jsonl`（選配：`data/train.parquet`）

## 結構說明
- 每列為一筆樣本：`{{"id": "...", "type": "...", "messages": [{{"role":"system","content":"..."}}, ...]}}`
- 訓練時可擷取第一個 `user` 與對應 `assistant` 形成 (instruction, response) pair，或直接使用 chat 格式的 trainer。

## 來源與限制
- Model: gemma-3-12b-it（Twinkle AI 社群）
- 語言：繁體中文
- 使用情境：教學示範用；不代表專業意見

## 授權
- 建議使用 **CC BY 4.0**；若另有需求請調整 `license` 欄位。
"""

CARD_PATH.write_text(card_md, encoding="utf-8")
print("✅ 產生 Dataset Card：", CARD_PATH.resolve())

In [ ]:
if ALSO_UPLOAD_PARQUET:
    from datasets import Dataset
    import pandas as pd

    # 讀 jsonl → Dataset → parquet
    rows = []
    with LOCAL_JSONL.open("r", encoding="utf-8") as f:
        for line in f:
            rows.append(json.loads(line))
    ds = Dataset.from_pandas(pd.DataFrame(rows))
    PARQUET_PATH = Path("outputs/train.parquet")
    ds.to_parquet(PARQUET_PATH)
    print("✅ 產生 parquet：", PARQUET_PATH.resolve())
else:
    PARQUET_PATH = None

In [ ]:
# HF 對部分副檔名會自動 LFS，但 .jsonl 有時未必；這裡顯式指定
GITATTR_PATH = Path("outputs/.gitattributes")
gitattributes = """*.jsonl filter=lfs diff=lfs merge=lfs -text
*.parquet filter=lfs diff=lfs merge=lfs -text
"""
GITATTR_PATH.write_text(gitattributes, encoding="utf-8")
print("✅ 產生 .gitattributes")

In [ ]:
# ==== 先建立（或覆用）Dataset repo ====
api = HfApi()
api.create_repo(
    repo_id=REPO_ID,
    repo_type="dataset",
    exist_ok=True,   # 已存在則不報錯
    private=False    # 需要私有可改 True
)

In [ ]:
# 建議的 Hub 目錄結構
REMOTE_JSONL = "data/train.jsonl"
REMOTE_PARQUET = "data/train.parquet" if PARQUET_PATH else None
REMOTE_CARD = "README.md"
REMOTE_GITATTR = ".gitattributes"

# 逐檔上傳（huggingface_hub 會自動處理 commit）
upload_file(
    path_or_fileobj=str(LOCAL_JSONL),
    path_in_repo=REMOTE_JSONL,
    repo_id=REPO_ID,
    repo_type="dataset",
)

upload_file(
    path_or_fileobj=str(CARD_PATH),
    path_in_repo=REMOTE_CARD,
    repo_id=REPO_ID,
    repo_type="dataset",
)

upload_file(
    path_or_fileobj=str(GITATTR_PATH),
    path_in_repo=REMOTE_GITATTR,
    repo_id=REPO_ID,
    repo_type="dataset",
)

if PARQUET_PATH and PARQUET_PATH.exists():
    upload_file(
        path_or_fileobj=str(PARQUET_PATH),
        path_in_repo=REMOTE_PARQUET,
        repo_id=REPO_ID,
        repo_type="dataset",
    )

print("✅ 上傳完成")
print(f"👉 瀏覽： https://huggingface.co/datasets/{REPO_ID}")